In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
p0_20p0 = np.array((1.14385, 13.28584, 0.34846, 11.30750, 11.97186, 0., 0.))
Npert = 200

In [3]:
def plot_fp(param,func,polyd2,polyd3,ip,jf,krun,folder):
    plt.figure()
    x = np.linspace(min(param),max(param),1000)
    plt.plot(x,polyd2[0]*x**2+polyd2[1]*x+polyd2[2],'r-')
    plt.plot(x,polyd3[0]*x**3+polyd3[1]*x**2+polyd3[2]*x+polyd3[3],'g-')
    plt.plot(param,func,'b.')
    plt.axvline(p0_20p0[ip],linestyle=':',color='grey')
    plt.savefig(folder+'/run{}_p{}f{}'.format(krun,ip,jf))
    plt.close()

In [4]:
def process_dfdp(p0_all,run,krun,plot_folder,plot=0):
    poly2dfdp = np.zeros((7,100))
    poly3dfdp = np.zeros((7,100))
    for i in range(7):
        p0 = p0_all[i]
        for j in range(100):
            polyd3 = np.polyfit(run['param'][i*Npert:i*Npert+Npert,i],\
                                     run['func_all'][i*Npert:i*Npert+Npert,j],3)
            poly3dfdp[i,j] = 3.*polyd3[0]*p0**2+2.*polyd3[1]*p0+polyd3[2]
            polyd2 = np.polyfit(run['param'][i*Npert:i*Npert+Npert,i],\
                                     run['func_all'][i*Npert:i*Npert+Npert,j],2)
            poly2dfdp[i,j] = 2.*polyd2[0]*p0+polyd2[1]
            if plot:
                plot_fp(run['param'][i*Npert:i*Npert+Npert,i],run['func_all'][i*Npert:i*Npert+Npert,j],polyd2,polyd3,\
                    i,j,krun,plot_folder)
    return poly2dfdp,poly3dfdp

In [5]:
run_97 = np.load('Run_061218/bolp_20p0_um_0_b_97.npz')

In [6]:
poly2_97,poly3_97 = process_dfdp(p0_20p0,run_97,97,'Run_061218/bolp20p0_um_0')

In [7]:
run_250 = np.load('Run_061218/bolp_20p0_um_0_b_250.npz')

In [8]:
poly2_250,poly3_250 = process_dfdp(p0_20p0,run_250,250,'Run_061218/bolp20p0_um_0')

In [9]:
run_455 = np.load('Run_061218/bolp_20p0_um_0_b_455.npz')

In [10]:
poly2_455,poly3_455 = process_dfdp(p0_20p0,run_455,455,'Run_061218/bolp20p0_um_0')

In [11]:
run_904 = np.load('Run_061218/bolp_20p0_um_0_b_904.npz')

In [12]:
poly2_904,poly3_904 = process_dfdp(p0_20p0,run_904,904,'Run_061218/bolp20p0_um_0')

In [13]:
run_63 = np.load('Run_061218/bolp_20p0_um_0_b_63.npz')

In [14]:
poly2_63,poly3_63 = process_dfdp(p0_20p0,run_63,63,'Run_061218/bolp20p0_um_0')

In [15]:
fid_20p0 = np.load('Run_061218/bolp_20p0_0.npz')

def calc_cov(covjk,func_real):
    return np.mean(covjk,axis=0)+np.cov(func_real.T)

cov = calc_cov(fid_20p0['func_all_cov'],fid_20p0['func_all'])

In [16]:
def calc_fisher(dfdp, covtot):
    fmatrix = np.zeros((7,7))
    for i in range(7):
        for j in range(7):
            fmatrix[i,j] = np.dot(dfdp[i],np.dot(np.linalg.inv(covtot),dfdp[j]))
    return fmatrix

def calc_1sigma(fisher):
    return np.sqrt(np.linalg.inv(fisher).diagonal())

def calc_1sigma_prior(fisher,i,priors2):
    fisher1 = fisher*1.
    fisher1[i,i] += priors2
    return np.sqrt(np.linalg.inv(fisher1).diagonal())

In [17]:
fisher2_97 = calc_fisher(poly2_97,cov)

print calc_1sigma(fisher2_97)

print calc_1sigma_prior(fisher2_97,2,100)

fisher3_97 = calc_fisher(poly3_97,cov)

print calc_1sigma(fisher3_97)

print calc_1sigma_prior(fisher3_97,2,100)

[ 0.02129043  0.01647229  0.03364061  0.36458296  0.01458007  0.19516275
  0.17375572]
[ 0.02128586  0.01647044  0.03188477  0.3642888   0.01401256  0.19402472
  0.1734755 ]
[ 0.01731084  0.01476871  0.02516169  0.37211377  0.01162991  0.11583793
  0.14189921]
[ 0.01729276  0.01476114  0.02440111  0.37210493  0.01143033  0.11563591
  0.1417909 ]


In [18]:
fisher2_250 = calc_fisher(poly2_250,cov)

print calc_1sigma(fisher2_250)

print calc_1sigma_prior(fisher2_250,2,100)

fisher3_250 = calc_fisher(poly3_250,cov)

print calc_1sigma(fisher3_250)

print calc_1sigma_prior(fisher3_250,2,100)

[ 0.02229637  0.01592145  0.03746581  0.36268746  0.0162205   0.16740746
  0.17045793]
[ 0.02226896  0.01591781  0.03508427  0.36234847  0.01538519  0.16687792
  0.17024803]
[ 0.01693361  0.01342484  0.02836483  0.3250653   0.01313871  0.10985709
  0.13166366]
[ 0.016915    0.01341041  0.02728831  0.32500601  0.01278743  0.10981375
  0.13162106]


In [19]:
fisher2_455 = calc_fisher(poly2_455,cov)

print calc_1sigma(fisher2_455)

print calc_1sigma_prior(fisher2_455,2,100)

fisher3_455 = calc_fisher(poly3_455,cov)

print calc_1sigma(fisher3_455)

print calc_1sigma_prior(fisher3_455,2,100)

[ 0.02445873  0.0186982   0.03590606  0.4421317   0.01641356  0.17413592
  0.16437149]
[ 0.02443872  0.01869808  0.03379367  0.44127918  0.01565919  0.17380358
  0.1640661 ]
[ 0.01641923  0.01286694  0.02585071  0.29339938  0.01251278  0.10669828
  0.14708358]
[ 0.01639976  0.01286102  0.02502797  0.29323479  0.01224445  0.1066946
  0.14700949]


In [20]:
fisher2_904 = calc_fisher(poly2_904,cov)

print calc_1sigma(fisher2_904)

print calc_1sigma_prior(fisher2_904,2,100)

fisher3_904 = calc_fisher(poly3_904,cov)

print calc_1sigma(fisher3_904)

print calc_1sigma_prior(fisher3_904,2,100)

[ 0.02007479  0.017792    0.03559769  0.41929009  0.01617471  0.18360482
  0.16203292]
[ 0.02006178  0.01779197  0.0335362   0.41848322  0.01543964  0.18321925
  0.16183406]
[ 0.01427017  0.01344586  0.02645954  0.28922201  0.01267793  0.11844159
  0.14494893]
[ 0.01425922  0.01344224  0.02557928  0.28895782  0.01237703  0.11842865
  0.14489698]


In [21]:
fisher2_63 = calc_fisher(poly2_63,cov)

print calc_1sigma(fisher2_63)

print calc_1sigma_prior(fisher2_63,2,100)

fisher3_63 = calc_fisher(poly3_63,cov)

print calc_1sigma(fisher3_63)

print calc_1sigma_prior(fisher3_63,2,100)

[ 0.02242577  0.01623523  0.03552856  0.37233165  0.01665841  0.17514403
  0.1672043 ]
[ 0.02239041  0.01623031  0.03347839  0.37148529  0.01589712  0.17485994
  0.16703843]
[ 0.01712541  0.01507718  0.02552509  0.34619234  0.0129712   0.11294109
  0.14263411]
[ 0.01709369  0.01504753  0.02473212  0.34617559  0.01271283  0.1128981
  0.14262837]


In [22]:
print calc_1sigma(calc_fisher(poly2_97[:,:20],cov[:20,:20]))
print calc_1sigma(calc_fisher(poly3_97[:,:20],cov[:20,:20]))

[ 0.05657078  0.04024382  0.299262    1.37330995  0.12380026  0.97420425
  0.58163499]
[ 0.08147108  0.04731124  0.23828823  2.26406881  0.11617577  0.77609848
  0.58365037]


In [23]:
print calc_1sigma(calc_fisher(poly2_250[:,:20],cov[:20,:20]))
print calc_1sigma(calc_fisher(poly3_250[:,:20],cov[:20,:20]))

[ 0.04477799  0.03761559  0.23955153  0.81842976  0.09076594  0.60199546
  0.5566405 ]
[ 0.03139477  0.02934999  0.20746957  0.67715942  0.07877322  0.4765552
  0.40527132]


In [24]:
print calc_1sigma(calc_fisher(poly2_455[:,:20],cov[:20,:20]))
print calc_1sigma(calc_fisher(poly3_455[:,:20],cov[:20,:20]))

[ 0.04971207  0.04436415  0.33478375  1.05709977  0.12488224  0.70583043
  0.41437518]
[ 0.03708281  0.03638567  0.17965873  0.82343149  0.06710745  0.5635757
  0.42768173]


In [25]:
print calc_1sigma(calc_fisher(poly2_904[:,:20],cov[:20,:20]))
print calc_1sigma(calc_fisher(poly3_904[:,:20],cov[:20,:20]))

[ 0.09386811  0.10303952  0.27582912  2.44932285  0.1088011   0.5784433
  0.47881634]
[ 0.04744794  0.0797207   0.17130334  1.72092848  0.07028593  0.50363953
  0.60883361]


In [26]:
print calc_1sigma(calc_fisher(poly2_63[:,:20],cov[:20,:20]))
print calc_1sigma(calc_fisher(poly3_63[:,:20],cov[:20,:20]))

[ 0.06112143  0.04522238  0.26324765  1.09464485  0.09197647  0.68629669
  0.46194056]
[ 0.03908473  0.03214427  0.20356526  0.79145509  0.07721344  0.63390733
  0.28067078]
